In [1]:
import sys
sys.path.append('../')

In [2]:
from src.data_loader import *

In [3]:
train = load_data_from_pickle('../data/intermediate/train.pkl')
test = load_data_from_pickle('../data/intermediate/test.pkl')

In [4]:
train['TicketPrefix'] = train['Ticket'].str.extract('([A-Za-z]+)', expand=False)
test['TicketPrefix'] = test['Ticket'].str.extract('([A-Za-z]+)', expand=False)

In [5]:
from sklearn.preprocessing import OneHotEncoder

In [6]:
train['TicketPattern'] = train['Ticket'].str.contains('[A-Za-z]').map({False: 'Digits', True: 'AlphaNumeric'})
test['TicketPattern'] = test['Ticket'].str.contains('[A-Za-z]').map({False: 'Digits', True: 'AlphaNumeric'})

In [7]:
train.drop('Ticket', axis=1, inplace=True)
test.drop('Ticket', axis=1, inplace=True)

In [8]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

encoded_features_train = encoder.fit_transform(train[['TicketPattern']])
train_encoded = pd.concat([train.drop('TicketPattern', axis=1), pd.DataFrame(encoded_features_train, columns=encoder.get_feature_names_out(['TicketPattern']))], axis=1)

encoded_features_test = encoder.transform(test[['TicketPattern']])
test_encoded = pd.concat([test.drop('TicketPattern', axis=1), pd.DataFrame(encoded_features_test, columns=encoder.get_feature_names_out(['TicketPattern']))], axis=1)

In [9]:
ticket_prefix_train = train[['TicketPrefix']]
ticket_prefix_test = test[['TicketPrefix']]

In [10]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_ticket_prefix_train = encoder.fit_transform(ticket_prefix_train)
encoded_ticket_prefix_test = encoder.transform(ticket_prefix_test)

In [11]:
encoded_ticket_prefix_train_df = pd.DataFrame(encoded_ticket_prefix_train, columns=encoder.get_feature_names_out(['TicketPrefix']))
encoded_ticket_prefix_test_df = pd.DataFrame(encoded_ticket_prefix_test, columns=encoder.get_feature_names_out(['TicketPrefix']))

In [12]:
train = pd.concat([train, encoded_ticket_prefix_train_df], axis=1)
test = pd.concat([test, encoded_ticket_prefix_test_df], axis=1)

In [13]:
train.drop(['TicketPrefix', 'TicketPattern'], axis=1, inplace=True)
test.drop(['TicketPrefix', 'TicketPattern'], axis=1, inplace=True)

In [14]:
save_data_as_pickle(train, '../data/processed/train.pkl')
save_data_as_pickle(test, '../data/processed/test.pkl')